# imports

In [1]:
from pathlib import Path

In [2]:
import bs4

In [3]:
from bs4 import BeautifulSoup as BS

In [4]:
from dataclasses import dataclass

In [5]:
from enum import Enum

In [6]:
from serde import serialize, deserialize
from serde.json import to_json

# Récupération du contenu html

On a **manuellement** sauvegarder les 7 premières pages correspondant aux résultats de la recherche *python* sur [amazon.fr](www.amazon.fr)

In [7]:
racine = Path(".").resolve()
print(racine)

D:\Cours Mecen (M2)\Machine Learning\ML-2023-2024\seance03


In [8]:
for contenu in racine.glob("*"):
    print(contenu)

D:\Cours Mecen (M2)\Machine Learning\ML-2023-2024\seance03\03.ipynb
D:\Cours Mecen (M2)\Machine Learning\ML-2023-2024\seance03\articles.json
D:\Cours Mecen (M2)\Machine Learning\ML-2023-2024\seance03\pages


In [9]:
pages = racine / "pages"
print(pages)

D:\Cours Mecen (M2)\Machine Learning\ML-2023-2024\seance03\pages


In [10]:
for chemin in pages.glob("*.html"):
    print(chemin)

D:\Cours Mecen (M2)\Machine Learning\ML-2023-2024\seance03\pages\amazon_p1.html
D:\Cours Mecen (M2)\Machine Learning\ML-2023-2024\seance03\pages\amazon_p2.html
D:\Cours Mecen (M2)\Machine Learning\ML-2023-2024\seance03\pages\amazon_p3.html
D:\Cours Mecen (M2)\Machine Learning\ML-2023-2024\seance03\pages\amazon_p4.html
D:\Cours Mecen (M2)\Machine Learning\ML-2023-2024\seance03\pages\amazon_p5.html
D:\Cours Mecen (M2)\Machine Learning\ML-2023-2024\seance03\pages\amazon_p6.html


In [11]:
htmls = list()
for chemin in pages.glob("*.html"):
    htmls.append(chemin.read_text(encoding="utf8"))

# Exploitation de la première page html

## Exercice

En utilisant les outils de développement du navigateur et `bs4`, récupérer la liste des éléments représentant des livres dans la première page.

In [12]:
soupe = BS(htmls[0])

In [13]:
result_items = soupe.find_all(name="div", attrs={"class": "s-result-item"})

In [14]:
len(result_items)

66

**ATTENTION** on voit visuellement qu'il n'y a que 60 éléments à récupérer dans la page, donc il y a des intrus à identifier.

In [15]:
result_item = result_items[6]

`.prettify()` permet de représenter la page `HTML` bien indentée

In [16]:
print(result_item.prettify())

<div class="sg-col-4-of-24 sg-col-4-of-12 s-result-item s-asin sg-col-4-of-16 sg-col s-widget-spacing-small sg-col-4-of-20" data-asin="2212134347" data-cel-widget="search_result_6" data-component-id="35" data-component-type="s-search-result" data-index="7" data-uuid="ae59066a-490b-440c-9fb4-bdcac883e08c">
 <div class="sg-col-inner">
  <div cel_widget_id="MAIN-SEARCH_RESULTS-7" class="s-widget-container s-spacing-small s-widget-container-height-small celwidget slot=MAIN template=SEARCH_RESULTS widgetId=search-results_6" data-cel-widget="MAIN-SEARCH_RESULTS-7" data-csa-c-id="n4rabc-757nxu-avzfy4-ki1ill" data-csa-c-item-id="amzn1.asin.1.2212134347" data-csa-c-pos="6" data-csa-c-type="item" data-csa-op-log-render="">
   <div class="s-card-container s-overflow-hidden aok-relative puis-expand-height puis-include-content-margin puis puis-v20azwp0smsgc01ytmkntf1rk7n s-latency-cf-section s-card-border">
    <div class="a-section a-spacing-base">
     <div class="s-product-image-container aok-re

In [17]:
result_item.find_all(name="span", attrs={"class": "a-text-normal"})

[<span class="a-size-base-plus a-color-base a-text-normal">Apprendre à programmer avec Python 3</span>]

In [18]:
result_item.find_all(name="span", attrs={"class": "a-price-whole"})

[<span class="a-price-whole">32,40</span>]

In [19]:
list_author_1 = [
    x.text 
    for x in result_item.select("span.a-size-base:not([class*= 'a-size-base '])") 
    if x.text[0].isupper()
] 

In [20]:
list_author_2 = [
    x.text 
    for x in result_item.find_all(
        name="a", attrs={"class": "a-size-base a-link-normal s-underline-text s-underline-link-text s-link-style"}
    )
    if "Format" not in x.text
]

In [21]:
list_author_1 + list_author_2

['Gérard Swinnen']

In [22]:
def recup_titre_prix(element: bs4.element.Tag) -> tuple[str, str]:
    titre, = element.find_all(name="span", attrs={"class": "a-text-normal"})
    prix, = element.find_all(name="span", attrs={"class": "a-price-whole"})
    return (titre.text, prix.text)
    

In [23]:
recup_titre_prix(result_item)

('Apprendre à programmer avec Python 3', '32,40')

In [24]:
extractions = list()
for item in result_items:
    try:
        extractions.append(recup_titre_prix(item))
    except ValueError:
        pass

In [25]:
# il y a normalement 60 livres dans la page
len(extractions)

56

In [26]:
# Inspection visuelle
for extraction in extractions:
    print(extraction)

('Python pour débutants: Programmer un réseau neuronal en 7 jours', '16,99')
("Apprendre\n Python: Un Cours Accéléré sur la Programmation Python et Comment \nCommencer à l’utiliser pour Coder. Apprenez les Bases de Machine \nLearning et de l'analyse de Données", '16,87')
("PYTHON: L'introduction et les fondements pour apprendre à programmer en python en 7 jours", '12,90')
('Python\n GUI Programming with PAGE: Create professional-looking GUIs for Python \napplications efficiently and effectively (English Edition)', '32,69')
('Python pour débutants: Programmer un réseau neuronal en 7 jours', '16,99')
('Apprendre à programmer avec Python 3', '32,40')
("J'apprends à programmer avec Python", '12,95')
("Programmer en Python - Apprendre la programmation de façon claire, concise et efficace - collection O'Reilly", '39,00')
("100\n Exercices Python pour s'entrainer: Un ensemble d'exercices avec \ndifférents niveaux de complexité | Débutant - Intermédiaire - Avancé | \nExercices corrigés pour to

## Exercice

Ajuster la recherche précédente pour récupérer exactement les résultats voulus.

In [27]:
for extraction in extractions:
    if "python" not in extraction[0].lower():
        print(extraction)

('Les algorithmes, c’est plus simple avec un dessin !', '19,90')


In [28]:
class ParsingError(Exception):
    pass

def recup_titre_prix(element: bs4.element.Tag) -> tuple[str, str]:
    try:
        spans = element.find_all(name="span", attrs={"class": "a-text-normal"})
        titre, = spans
        titre = titre.text
    except ValueError:
        raise ParsingError
    try:
        spans = element.find_all(name="span", attrs={"class": "a-price-whole"})
        prix, = spans
        prix = prix.text
    except ValueError:
        raise ParsingError
    if "python" not in titre.lower():
        raise ParsingError
    return (titre, prix)

In [29]:
extractions = list()
for item in result_items:
    try:
        extractions.append(recup_titre_prix(item))
    except ParsingError:
        pass

In [30]:
len(extractions)

55

# Assemblage

In [31]:
class ParsingError(Exception):
    pass

def recup_titre_prix(element: bs4.element.Tag) -> tuple[str, str]:
    try:
        spans = element.find_all(
            name="span", 
            attrs={"class": "a-text-normal"}
        )
        titre, = spans
        titre = titre.text
    except ValueError:
        raise ParsingError
    try:
        spans = element.find_all(
            name="span", 
            attrs={"class": "a-price-whole"}
        )
        prix, = spans
        prix = prix.text
    except ValueError:
        raise ParsingError
    if "python" not in titre.lower():
        raise ParsingError
    return (titre, prix)

In [32]:
extractions = list()
for html in htmls:
    soupe = BS(html, features="lxml")
    result_items = soupe.find_all(
        name="div", 
        attrs={"class": "s-result-item"}
    )
    for item in result_items:
        try:
            extractions.append(recup_titre_prix(item))
        except ParsingError:
            pass

In [33]:
len(extractions)

317

## Exercice

Enrichir le code pour extraire quand c'est possible:
- les auteurs
- l'évalution moyenn
- le nombres d'évaluations
- broché ou relié ou...
- sponsorisé ou pas?

In [34]:
@serialize
@dataclass
class Article:
    titre: str
    prix: float
    auteurs: list
    evaluation: float
    nombre_evaluations: int
    sponsorise: bool

In [35]:
def get_title(element: bs4.element.Tag) -> str:
    try:
        spans = element.find_all(
            name="span", 
            attrs={"class": "a-text-normal"}
        )
        title, = spans
        title = title.text
    except ValueError:
        raise ParsingError
    if "python" not in title.lower():
        raise ParsingError
    return title

In [36]:
def get_price(element: bs4.element.Tag) -> str:
    try:
        spans = element.find_all(
            name="span", 
            attrs={"class": "a-price-whole"}
        )
        price, = spans
        price = float(price.text.replace(",", "."))
    except ValueError:
        raise ParsingError
    return price

In [37]:
def get_authors(element: bs4.element.Tag) -> list:
    constraints = ["Format", "Relié", "Broché", "Livres audio Audible"]
    list_author_1 = [
        x.text 
        for x in element.select("span.a-size-base:not([class*= 'a-size-base '])") 
        if x.text[0].isupper()
    ]
    list_author_2 = [
        x.text 
        for x in element.find_all(
            name="a", attrs={"class": "a-size-base a-link-normal s-underline-text s-underline-link-text s-link-style"}
        )
        if all(constraint not in x.text for constraint in constraints)
    ]
    authors = list_author_1 + list_author_2
    return authors

In [38]:
def get_rating(element: bs4.element.Tag) -> float:
    """a-icon-alt"""
    try:
        spans = element.find_all(
            name="span", 
            attrs={"class": "a-size-base puis-normal-weight-text"}
        )
        rating, = spans
        rating = float(rating.text.replace(",", "."))
    except ValueError:
        raise ParsingError
    return rating

In [39]:
def get_rating_number(element: bs4.element.Tag) -> int:
    """_summary_

    Args:
        element (bs4.element.Tag): _description_

    Raises:
        ParsingError: _description_

    Returns:
        int: _description_
    """
    try:
        spans = element.find_all(
            name="span", 
            attrs={"class": "a-size-base s-underline-text"}
        )
        nb_rating, = spans
        nb_rating = int(nb_rating.text)
    except ValueError:
        raise ParsingError
    return nb_rating
    

In [40]:
def get_sponsored(element: bs4.element.Tag) -> bool:
    spans = element.find_all(
        name="span",
        attrs={"class": "a-color-base"}
    )
    for span in spans:
        if span.text == "Sponsorisé":
            return True
    return False

In [41]:
def article_generation(element: bs4.element.Tag) -> Article:
    titre = get_title(element)
    prix = get_price(element)
    auteurs = get_authors(element)
    evaluation = get_rating(element)
    nombre_evaluations = get_rating_number(element)
    sponsorise = get_sponsored(element)
    return Article(titre=titre, prix=prix, auteurs=auteurs, evaluation=evaluation, nombre_evaluations=nombre_evaluations,sponsorise=sponsorise)

In [42]:
articles: list[Article] = list()
for html in htmls:
    soupe = BS(html, features="lxml")
    result_items = soupe.find_all(
        name="div", 
        attrs={"class": "s-result-item"}
    )
    for item in result_items:
        try:
            articles.append(article_generation(item)) 
        except ParsingError:
            pass

In [43]:
articles_json = to_json(articles)

In [47]:
articles

[Article(titre='Python pour débutants: Programmer un réseau neuronal en 7 jours', prix=16.99, auteurs=['Violette Chiara', 'Benjamin Spahic'], evaluation=4.6, nombre_evaluations=65, sponsorise=True),
 Article(titre="Apprendre\n Python: Un Cours Accéléré sur la Programmation Python et Comment \nCommencer à l’utiliser pour Coder. Apprenez les Bases de Machine \nLearning et de l'analyse de Données", prix=16.87, auteurs=['Damon Parker'], evaluation=4.9, nombre_evaluations=10, sponsorise=True),
 Article(titre="PYTHON: L'introduction et les fondements pour apprendre à programmer en python en 7 jours", prix=12.9, auteurs=['Jack Fellers'], evaluation=3.8, nombre_evaluations=102, sponsorise=True),
 Article(titre='Python\n GUI Programming with PAGE: Create professional-looking GUIs for Python \napplications efficiently and effectively (English Edition)', prix=32.69, auteurs=['Gregory Walters'], evaluation=5.0, nombre_evaluations=1, sponsorise=True),
 Article(titre='Python pour débutants: Programm

In [48]:
file_path = "articles.json"

with open(file_path, "w",  encoding='utf-8') as json_file:
    json_file.write(articles_json)

## Exercice

Sérialiser vers json `articles`.

In [49]:
import polars as pl

In [51]:
pl.read_json("articles.json")

titre,prix,auteurs,evaluation,nombre_evaluations,sponsorise
str,f64,list[str],f64,i64,bool
"""Python pour dé…",16.99,"[""Violette Chiara"", ""Benjamin Spahic""]",4.6,65,true
"""Apprendre Pyt…",16.87,"[""Damon Parker""]",4.9,10,true
"""PYTHON: L'intr…",12.9,"[""Jack Fellers""]",3.8,102,true
"""Python GUI Pr…",32.69,"[""Gregory Walters""]",5.0,1,true
"""Python pour dé…",16.99,"[""Violette Chiara"", ""Benjamin Spahic""]",4.6,65,false
"""Apprendre à pr…",32.4,"[""Gérard Swinnen""]",4.6,327,false
"""J'apprends à p…",12.95,"[""Louie Stowell"", ""Rosie Dickins""]",4.5,493,false
"""Programmer en …",39.0,"[""Luciano Ramalho""]",4.4,93,false
"""100 Exercices…",12.99,"[""Laurentine K.Masson""]",4.4,40,false
